# RA2CE feature: Single link losses analysis

## Single link losses Analysis

In [2]:
from pathlib import Path
from ra2ce.ra2ce_handler import Ra2ceHandler
import geopandas as gpd

# Import config data
root_dir = Path("data", "single_link_losses")
assert root_dir.exists()

# Load network data.
from ra2ce.network.network_config_data.network_config_data_reader import NetworkConfigDataReader
_network_config_data = NetworkConfigDataReader().read(root_dir.joinpath("network.ini"))

# Load analysis data.
from ra2ce.analysis.analysis_config_data.analysis_config_data_reader import AnalysisConfigDataReader
_analysis_config_data = AnalysisConfigDataReader().read(root_dir.joinpath("analysis.ini"))

### Modify the analysis config data with custom properties

In [3]:
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionLosses

# Modify the first analysis (assuming it's the only one)
_found_analysis: AnalysisSectionLosses = _analysis_config_data.analyses[0]
_input_analysis_path = root_dir.joinpath("input_analysis_data")
_found_analysis.traffic_intensities_file = _input_analysis_path.joinpath("traffic_intensities.csv")
_found_analysis.resilience_curves_file = _input_analysis_path.joinpath("resilience_curve.csv")
_found_analysis.values_of_time_file = _input_analysis_path.joinpath("values_of_time.csv")

### Configure network

Let's have a look first at the hazard overlay to locate the road segments for which losses will be expected. We run only the network configuration.

In [4]:
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)
handler.configure()

In [5]:
hazard_output = root_dir / "static" / "output_graph" / "base_graph_hazard_edges.gpkg"
hazard_gdf = gpd.read_file(hazard_output, driver = "GPKG")
hazard_gdf.explore(column="EV1_ma", cmap = "viridis")

### Run losses analysis

In [10]:
handler.run_analysis()

c:\Users\hauth\anaconda3\envs\ra2ce_env\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


[AnalysisResultWrapper(analysis_result=      ID    u    v    EV1_fr    EV1_ma  \
 0     86    2   42  0.014318  1.369213   
 1     30   29   31  0.079975  0.345044   
 2    100   30   82  0.100808  0.654458   
 3     85   42   73  0.027676  0.409945   
 4    264   46   51  0.002634  0.766717   
 ..   ...  ...  ...       ...       ...   
 225  253  153  156  0.000000  0.000000   
 226  246  154  155  0.000000  0.000000   
 227  249  154  156  0.000000  0.000000   
 228  248  156  157  0.000000  0.000000   
 229  257  157  158  0.000000  0.000000   
 
                                              alt_nodes  alt_time  avgspeed  \
 0    [2, 0, 11, 12, 10, 7, 6, 9, 75, 121, 118, 5, 3...  0.099722      59.0   
 1                                     [29, 30, 82, 31]  0.009356      59.0   
 2                                     [30, 29, 31, 82]  0.005288      30.0   
 3                                                  nan       NaN      60.0   
 4                                [46, 48, 47, 16

Let's inspect the results of the analysis. For every segment in the network, the losses are expressed in vehicle loss hour (vlh) for all the types of trip defined in the input files: business, commute, freight and other.

In [15]:
analysis_output_folder = root_dir / "output" / "single_link_losses" # specify path to output folder

losses_gdf = gpd.read_file(analysis_output_folder/"beira_single_losses.gpkg") #specify the name of the geopackage holding your results (can be found in the analysis output folder)
losses_gdf.head() #display the attributes of the file

,ID,u,v,EV1_fr,EV1_ma,alt_nodes,alt_time,avgspeed,bridge,detour,...,node_B,rfid,rfid_c,time,vlh_business_EV1_ma,vlh_commute_EV1_ma,vlh_freight_EV1_ma,vlh_other_EV1_ma,vlh_EV1_ma_total,geometry
0,86,2,42,0.014318,1.369213,"[2, 0, 11, 12, 10, 7, 6, 9, 75, 121, 118, 5, 3...",0.099722,59.0,yes,1,...,42,125,127,0.043763,43.312500,30.333333,577.500000,513.333333,1164.479167,"LINESTRING (34.86702 -19.83164, 34.86711 -19.8..."
1,30,29,31,0.079975,0.345044,"[29, 30, 82, 31]",0.009356,59.0,yes,1,...,31,91,93,0.003898,0.515625,0.375000,0.500000,3.583333,4.973958,"LINESTRING (34.83997 -19.83592, 34.84019 -19.8..."
2,100,30,82,0.100808,0.654458,"[30, 29, 31, 82]",0.005288,30.0,yes,1,...,82,233,241,0.007967,-4.078125,-1.843750,-8.125000,-10.625000,-24.671875,"LINESTRING (34.84149 -19.83431, 34.84137 -19.8..."
3,85,42,73,0.027676,0.409945,nan,NaN,60.0,yes,0,...,73,208,216,0.016917,106.000000,117.000000,56.000000,960.000000,1239.000000,"LINESTRING (34.87145 -19.82349, 34.87126 -19.8..."
4,264,46,51,0.002634,0.766717,"[46, 48, 47, 160, 51]",0.008829,60.0,nan,1,...,51,151,153,0.002183,3.978516,1.312500,8.020833,6.927083,20.238932,"LINESTRING (34.85650 -19.83019, 34.85714 -19.8..."


The map below displays the total loss for all trip types per link

In [14]:
losses_roads_map = losses_gdf.explore(column='vlh_EV1_ma_total', tiles="CartoDB positron",)
display(losses_roads_map)